# Lezione 22: Clustering Gerarchico

## 🎯 Obiettivi della Lezione

K-Means richiede di specificare K a priori. Ma cosa succede se non abbiamo idea di quanti cluster esistono? Il **Clustering Gerarchico** costruisce una struttura ad albero che mostra tutte le possibili partizioni dei dati.

### Cosa Imparerai

| Concetto | Descrizione |
|----------|-------------|
| **Clustering Agglomerativo** | Approccio bottom-up: parti dai singoli punti |
| **Dendrogramma** | Visualizzazione ad albero della gerarchia |
| **Linkage Methods** | Single, Complete, Average, Ward |
| **Taglio del dendrogramma** | Come scegliere K a posteriori |

### Prerequisiti
- ✅ Lezione 20: K-Means Clustering
- ✅ Lezione 21: Scelta del numero di cluster
- ✅ Concetto di distanza euclidea

### Outcome
Alla fine saprai:
1. Applicare AgglomerativeClustering di sklearn
2. Creare e interpretare dendrogrammi
3. Scegliere il linkage appropriato per i tuoi dati
4. Decidere K "tagliando" il dendrogramma
5. Confrontare clustering gerarchico vs K-Means

---

```python
# Librerie necessarie
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.datasets import make_blobs
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist
```

---

## 1. Teoria: Clustering Gerarchico

### 1.1 Due Approcci: Agglomerativo vs Divisivo

```
AGGLOMERATIVO (Bottom-Up)          DIVISIVO (Top-Down)
──────────────────────────         ──────────────────────────
     ●──●                               ┌───────────┐
    /    \                              │ Tutti i   │
   ●      ●──●                          │  punti    │
  / \    /    \                         └─────┬─────┘
 ●   ●  ●      ●                              │
 │   │  │      │                         ┌────┴────┐
 A   B  C      D                         ▼         ▼
                                       ┌───┐     ┌───┐
Parte dai punti singoli               │A,B│     │C,D│
Li unisce progressivamente            └───┘     └───┘
```

**Agglomerativo (il più comune):**
1. Ogni punto è un cluster
2. Ad ogni passo, unisci i 2 cluster più simili
3. Ripeti fino ad avere 1 solo cluster

**Divisivo:**
1. Tutti i punti sono un cluster
2. Dividi il cluster in 2 parti
3. Ripeti ricorsivamente

> 📌 **sklearn implementa solo l'approccio agglomerativo** (più efficiente)

---

### 1.2 Il Dendrogramma

Il **dendrogramma** è la visualizzazione fondamentale del clustering gerarchico.

```
Distanza
   │
 4 │                    ┌────────┴────────┐
   │                    │                 │
 3 │           ┌────────┴──┐              │
   │           │           │              │
 2 │     ┌─────┴───┐       │         ┌────┴────┐
   │     │         │       │         │         │
 1 │  ┌──┴──┐      │       │      ┌──┴──┐      │
   │  │     │      │       │      │     │      │
 0 ├──A─────B──────C───────D──────E─────F──────G──
```

**Come leggerlo:**
- **Asse Y**: distanza/dissimilarità a cui avviene l'unione
- **Linee orizzontali**: cluster uniti a quella distanza
- **Altezza della linea**: quanto dissimili sono i cluster uniti
- **Taglio orizzontale**: determina il numero di cluster

**Tagliare il dendrogramma:**
- Taglio alto → pochi cluster (più generali)
- Taglio basso → molti cluster (più specifici)
- Cerca il taglio dove le "gambe" sono più lunghe

---

### 1.3 Metodi di Linkage: Come Misurare la Distanza tra Cluster

Quando unisci due cluster, come definisci la "distanza" tra gruppi di punti?

| Linkage | Distanza tra Cluster | Pro | Contro |
|---------|---------------------|-----|--------|
| **Single** | Minima tra tutte le coppie | Trova forme allungate | Effetto "catena" |
| **Complete** | Massima tra tutte le coppie | Cluster compatti | Sensibile a outlier |
| **Average** | Media di tutte le coppie | Bilanciato | Meno interpretabile |
| **Ward** | Minimizza varianza totale | Cluster sferici, bilanciati | Assume sfericità |

**Formule:**

$$d_{\text{single}}(A, B) = \min_{a \in A, b \in B} d(a, b)$$

$$d_{\text{complete}}(A, B) = \max_{a \in A, b \in B} d(a, b)$$

$$d_{\text{average}}(A, B) = \frac{1}{|A| \cdot |B|} \sum_{a \in A} \sum_{b \in B} d(a, b)$$

$$d_{\text{ward}}(A, B) = \sqrt{\frac{2|A||B|}{|A|+|B|}} \|\bar{a} - \bar{b}\|$$

> 📌 **Regola pratica:** Usa **Ward** come default (simile a K-Means)

---

### 1.4 Visualizzazione dei Metodi di Linkage

```
SINGLE LINKAGE                      COMPLETE LINKAGE
(nearest neighbor)                  (farthest neighbor)
                                    
    ○ ○                                 ○ ○
   ○   ○                               ○   ○
  ○  ↔  ○  ← distanza minima          ○  ↔  ○  ← distanza massima
 ○       ○                           ○       ○
                                    
Cluster A   Cluster B               Cluster A   Cluster B

→ Tende a creare cluster            → Tende a creare cluster
  allungati ("catene")                 compatti e sferici


AVERAGE LINKAGE                     WARD LINKAGE
(group average)                     (minimum variance)
                                    
    ○ ○                                 ○ ○
   ○ ═══ ○   media di                  ○   ○
  ○ ═════ ○   tutte le                ○ ⊕ ○  ← minimizza
 ○ ═══════ ○   distanze              ○ ⊕   ○   varianza interna
                                    
→ Bilanciato                        → Cluster bilanciati
  compromesso                         simili a K-Means
```

---

### 1.5 Gerarchico vs K-Means: Confronto

| Aspetto | K-Means | Gerarchico |
|---------|---------|------------|
| **K richiesto a priori** | ✅ Sì | ❌ No (decidibile a posteriori) |
| **Complessità** | O(n·K·i) | O(n²) o O(n³) |
| **Scalabilità** | Buona per grandi dataset | Problematica per n > 10k |
| **Forma cluster** | Sferici | Dipende dal linkage |
| **Determinismo** | No (random init) | Sì |
| **Visualizzazione** | Scatter plot | Dendrogramma |
| **Interpretabilità** | Centroidi | Gerarchia |

**Quando usare Gerarchico:**
- Dataset piccoli/medi (n < 10.000)
- Vuoi esplorare diverse granularità
- La gerarchia ha significato (es. tassonomia)
- Non sai quanti cluster cercare

**Quando usare K-Means:**
- Dataset grandi
- Sai (circa) quanti cluster vuoi
- Vuoi cluster sferici/compatti
- Hai bisogno di velocità

---

## 2. Schema Mentale: Workflow Clustering Gerarchico

```
                         DATI SCALATI
                              │
                              ▼
                    ┌─────────────────────┐
                    │ Calcola Linkage     │
                    │ (scipy.linkage)     │
                    └──────────┬──────────┘
                               │
                    ┌──────────▼──────────┐
                    │ Visualizza          │
                    │ Dendrogramma        │
                    └──────────┬──────────┘
                               │
                    ┌──────────▼──────────┐
                    │ Scegli altezza      │
                    │ di taglio           │
                    └──────────┬──────────┘
                               │
            ┌──────────────────┼──────────────────┐
            ▼                  ▼                  ▼
    scipy.fcluster    o    sklearn.Agglomerative
    (da linkage)           Clustering(n_clusters=K)
            │                                     │
            └──────────────────┬──────────────────┘
                               │
                    ┌──────────▼──────────┐
                    │ Valutazione         │
                    │ (Silhouette, etc.)  │
                    └─────────────────────┘
```

### Checklist

1. ☐ StandardScaler applicato
2. ☐ Dendrogramma visualizzato
3. ☐ Linkage scelto (Ward default)
4. ☐ Altezza di taglio identificata
5. ☐ Cluster estratti
6. ☐ Silhouette calcolata
7. ☐ Cluster interpretati

---

## 3. Demo Pratiche

### Demo 1: Primo Dendrogramma

Generiamo dati semplici e visualizziamo il dendrogramma per capire la struttura.

In [ ]:
# ============================================
# DEMO 1: Primo Dendrogramma
# ============================================

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.datasets import make_blobs
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

print("="*70)
print("DEMO 1: Il Tuo Primo Dendrogramma")
print("="*70)

# Generiamo un dataset semplice con 3 cluster ben separati
np.random.seed(42)
X, y_true = make_blobs(n_samples=30, centers=3, cluster_std=0.8, random_state=42)

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"\n📊 Dataset: {X.shape[0]} punti, {X.shape[1]} feature")
print(f"🎯 Cluster veri: 3")

# ============================================
# Calcolo del linkage con scipy
# ============================================
# linkage() restituisce una matrice con la storia delle fusioni
Z = linkage(X_scaled, method='ward')

print(f"\n📐 Matrice linkage shape: {Z.shape}")
print("   Ogni riga = una fusione: [cluster1, cluster2, distanza, n_punti]")

# ============================================
# Visualizzazione
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Dati originali
scatter = axes[0].scatter(X_scaled[:, 0], X_scaled[:, 1], c=y_true, 
                           cmap='viridis', s=100, edgecolors='black')
for i in range(len(X_scaled)):
    axes[0].annotate(str(i), (X_scaled[i, 0]+0.1, X_scaled[i, 1]+0.1), fontsize=8)
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')
axes[0].set_title('Dati originali (con indici)')
axes[0].grid(True, alpha=0.3)

# Plot 2: Dendrogramma
dendrogram(Z, ax=axes[1], leaf_rotation=90, leaf_font_size=8)
axes[1].set_xlabel('Indice del punto')
axes[1].set_ylabel('Distanza (Ward)')
axes[1].set_title('Dendrogramma')

# Linea di taglio suggerita
axes[1].axhline(y=5, color='red', linestyle='--', linewidth=2, label='Taglio suggerito')
axes[1].legend()

plt.tight_layout()
plt.show()

# ============================================
# Interpretazione
# ============================================
print("\n" + "="*70)
print("📊 COME LEGGERE IL DENDROGRAMMA")
print("="*70)
print("""
1. ASSE X: Indici dei punti originali (riordinati per vicinanza)

2. ASSE Y: Distanza a cui avviene la fusione
   - Fusioni basse = punti molto simili
   - Fusioni alte = cluster molto diversi

3. LINEE VERTICALI: Cluster che vengono uniti
   - L'altezza indica quanto sono dissimili

4. TAGLIO ORIZZONTALE (linea rossa):
   - Tagliando a y=5, otteniamo 3 cluster
   - Conta quante linee verticali attraversi

5. "GAMBE LUNGHE": 
   - Indicano separazioni naturali nei dati
   - Buoni punti di taglio
""")

---

### Demo 2: Confronto dei Metodi di Linkage

Vediamo come diversi metodi di linkage producono dendrogrammi (e cluster) diversi.

In [ ]:
# ============================================
# DEMO 2: Confronto Metodi di Linkage
# ============================================

print("="*70)
print("DEMO 2: Confronto Metodi di Linkage")
print("="*70)

# Usiamo gli stessi dati scalati
linkage_methods = ['single', 'complete', 'average', 'ward']

fig, axes = plt.subplots(2, 4, figsize=(18, 10))

for idx, method in enumerate(linkage_methods):
    # Calcola linkage
    Z = linkage(X_scaled, method=method)
    
    # Estrai cluster con fcluster (taglio a 3 cluster)
    labels = fcluster(Z, t=3, criterion='maxclust')
    
    # Calcola silhouette
    sil_score = silhouette_score(X_scaled, labels)
    
    # Riga 1: Dendrogrammi
    dendrogram(Z, ax=axes[0, idx], leaf_rotation=90, leaf_font_size=6,
               truncate_mode='lastp', p=12)
    axes[0, idx].set_title(f'{method.upper()}\nLinkage', fontsize=12)
    axes[0, idx].set_xlabel('Cluster')
    axes[0, idx].set_ylabel('Distanza')
    
    # Riga 2: Scatter con cluster risultanti
    scatter = axes[1, idx].scatter(X_scaled[:, 0], X_scaled[:, 1], 
                                    c=labels, cmap='viridis', s=80, 
                                    edgecolors='black', alpha=0.8)
    axes[1, idx].set_xlabel('Feature 1')
    axes[1, idx].set_ylabel('Feature 2')
    axes[1, idx].set_title(f'Silhouette: {sil_score:.3f}')
    axes[1, idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================
# Analisi dei risultati
# ============================================
print("\n" + "="*70)
print("📊 CONFRONTO METODI")
print("="*70)

print(f"\n{'Metodo':<12} {'Silhouette':<12} {'Note'}")
print("-" * 50)

for method in linkage_methods:
    Z = linkage(X_scaled, method=method)
    labels = fcluster(Z, t=3, criterion='maxclust')
    sil = silhouette_score(X_scaled, labels)
    
    if method == 'single':
        note = "Può creare catene"
    elif method == 'complete':
        note = "Cluster compatti"
    elif method == 'average':
        note = "Bilanciato"
    else:
        note = "Simile a K-Means ✓"
    
    print(f"{method:<12} {sil:<12.3f} {note}")

print("""
💡 OSSERVAZIONI:
   - WARD spesso dà risultati migliori per cluster sferici
   - SINGLE può unire punti lontani se c'è una "catena" di connessione
   - Per questo dataset, tutti i metodi funzionano bene (cluster ben separati)
""")

---

### Demo 3: L'Effetto Catena del Single Linkage

Vediamo quando il Single Linkage fallisce: con dati che formano una "catena".

In [ ]:
# ============================================
# DEMO 3: Effetto Catena del Single Linkage
# ============================================

print("="*70)
print("DEMO 3: L'Effetto Catena del Single Linkage")
print("="*70)

# Creiamo dati con un "ponte" tra due cluster
np.random.seed(42)

# Cluster 1 e 2 separati
cluster1 = np.random.randn(30, 2) + np.array([-3, 0])
cluster2 = np.random.randn(30, 2) + np.array([3, 0])

# Ponte: punti che collegano i due cluster
ponte = np.array([[-2, 0], [-1, 0], [0, 0], [1, 0], [2, 0]])

X_chain = np.vstack([cluster1, cluster2, ponte])
X_chain_scaled = StandardScaler().fit_transform(X_chain)

print(f"📊 Dataset con 'ponte' tra cluster")
print(f"   Cluster 1: 30 punti, Cluster 2: 30 punti, Ponte: 5 punti")

# ============================================
# Confronto Single vs Ward
# ============================================
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for idx, method in enumerate(['single', 'ward']):
    # Dendrogramma
    Z = linkage(X_chain_scaled, method=method)
    dendrogram(Z, ax=axes[0, idx], truncate_mode='lastp', p=15,
               leaf_rotation=90, leaf_font_size=8)
    axes[0, idx].set_title(f'Dendrogramma: {method.upper()}')
    axes[0, idx].set_xlabel('Cluster')
    axes[0, idx].set_ylabel('Distanza')
    
    # Cluster con K=2
    labels = fcluster(Z, t=2, criterion='maxclust')
    
    # Scatter
    colors = ['red' if l == 1 else 'blue' for l in labels]
    axes[1, idx].scatter(X_chain_scaled[:, 0], X_chain_scaled[:, 1], 
                          c=colors, s=60, alpha=0.7, edgecolors='black')
    
    # Evidenzia il ponte
    axes[1, idx].scatter(X_chain_scaled[-5:, 0], X_chain_scaled[-5:, 1],
                          c='green', s=150, marker='s', edgecolors='black',
                          label='Punti ponte', zorder=5)
    
    sil = silhouette_score(X_chain_scaled, labels)
    axes[1, idx].set_title(f'{method.upper()}: Silhouette={sil:.3f}')
    axes[1, idx].legend()
    axes[1, idx].grid(True, alpha=0.3)

# Terza colonna: dati originali
axes[0, 2].scatter(X_chain_scaled[:-5, 0], X_chain_scaled[:-5, 1], 
                    c='gray', s=60, alpha=0.5, label='Cluster')
axes[0, 2].scatter(X_chain_scaled[-5:, 0], X_chain_scaled[-5:, 1],
                    c='green', s=150, marker='s', edgecolors='black',
                    label='Ponte')
axes[0, 2].set_title('Dati originali con ponte')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

axes[1, 2].text(0.5, 0.5, 
"""EFFETTO CATENA
    
Il Single Linkage può unire 
cluster distanti se esistono 
punti "ponte" che li collegano.

SINGLE: unisce basandosi sulla
distanza minima, quindi il ponte
"inganna" l'algoritmo.

WARD: considera la varianza
totale, quindi resiste meglio
al ponte.""", 
                transform=axes[1, 2].transAxes, fontsize=11,
                verticalalignment='center', horizontalalignment='center',
                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\n💡 LEZIONE: Il Single Linkage è vulnerabile a punti che 'collegano' cluster!")
print("   Usa Ward o Complete per evitare questo problema.")

---

### Demo 4: Taglio del Dendrogramma e Scelta di K

Come usare il dendrogramma per scegliere il numero di cluster.

In [ ]:
# ============================================
# DEMO 4: Taglio del Dendrogramma
# ============================================

print("="*70)
print("DEMO 4: Come Scegliere K dal Dendrogramma")
print("="*70)

# Dataset più interessante con 4 cluster di dimensioni diverse
np.random.seed(42)
X_multi, _ = make_blobs(n_samples=100, centers=4, cluster_std=[0.8, 1.0, 0.6, 1.2],
                        center_box=(-10, 10), random_state=42)
X_multi_scaled = StandardScaler().fit_transform(X_multi)

# Calcolo linkage
Z = linkage(X_multi_scaled, method='ward')

# ============================================
# Metodo 1: Cercare le "gambe lunghe"
# ============================================
print("\n📐 Metodo 1: Cercare le gambe più lunghe")

# Le distanze delle fusioni sono nella terza colonna di Z
distances = Z[:, 2]

# Differenze tra distanze consecutive (accelerazione)
diff_distances = np.diff(distances)

# Le fusioni con il salto più grande indicano separazioni naturali
top_jumps_idx = np.argsort(diff_distances)[-5:][::-1]

print(f"   Top 5 salti nelle distanze:")
for i, idx in enumerate(top_jumps_idx):
    n_clusters = len(X_multi_scaled) - idx - 1
    print(f"   {i+1}. Dopo fusione {idx}: salto={diff_distances[idx]:.2f} → K={n_clusters}")

# ============================================
# Visualizzazione con tagli multipli
# ============================================
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Dendrogramma con linee di taglio
dendrogram(Z, ax=axes[0, 0], truncate_mode='lastp', p=20,
           leaf_rotation=90, leaf_font_size=8)
axes[0, 0].set_title('Dendrogramma con tagli', fontsize=12)

# Linee di taglio per K=2, 3, 4, 5
cut_heights = [18, 12, 8, 5]
colors_cut = ['red', 'orange', 'green', 'purple']
for height, color, k in zip(cut_heights, colors_cut, [2, 3, 4, 5]):
    axes[0, 0].axhline(y=height, color=color, linestyle='--', 
                        linewidth=2, label=f'K={k}')
axes[0, 0].legend(loc='upper right')

# Plot delle distanze di fusione
axes[0, 1].plot(range(len(distances)), distances, 'b-o', markersize=4)
axes[0, 1].set_xlabel('Indice fusione')
axes[0, 1].set_ylabel('Distanza')
axes[0, 1].set_title('Distanze delle fusioni')
axes[0, 1].grid(True, alpha=0.3)

# Plot dei salti
axes[0, 2].bar(range(len(diff_distances)), diff_distances, alpha=0.7)
axes[0, 2].set_xlabel('Indice fusione')
axes[0, 2].set_ylabel('Salto (differenza)')
axes[0, 2].set_title('Salti nelle distanze\n(le barre alte indicano separazioni naturali)')
axes[0, 2].grid(True, alpha=0.3)

# Scatter per K=2, 3, 4
for idx, k in enumerate([2, 3, 4]):
    labels = fcluster(Z, t=k, criterion='maxclust')
    sil = silhouette_score(X_multi_scaled, labels)
    
    scatter = axes[1, idx].scatter(X_multi_scaled[:, 0], X_multi_scaled[:, 1],
                                    c=labels, cmap='viridis', s=50, 
                                    alpha=0.7, edgecolors='black')
    axes[1, idx].set_title(f'K={k}, Silhouette={sil:.3f}')
    axes[1, idx].set_xlabel('Feature 1')
    axes[1, idx].set_ylabel('Feature 2')
    axes[1, idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================
# Riepilogo
# ============================================
print("\n" + "="*70)
print("📊 COME SCEGLIERE K DAL DENDROGRAMMA")
print("="*70)
print("""
1. GAMBE LUNGHE: Cerca i punti dove le linee verticali sono più alte
   → Indicano separazioni naturali nei dati

2. SALTI NELLE DISTANZE: Cerca i picchi nel grafico delle differenze
   → Il numero di cluster = n_punti - indice_salto - 1

3. CONFRONTO SILHOUETTE: Prova diversi K e confronta
   → Scegli quello con silhouette più alta

4. INTERPRETABILITÀ: Considera se i cluster hanno senso
   → A volte un K leggermente peggiore è più utile

💡 Per questo dataset: K=4 ha i cluster veri, ma K=3 ha silhouette simile
   → La scelta dipende dal contesto!
""")

---

### Demo 5: sklearn AgglomerativeClustering vs scipy

Confrontiamo i due approcci: scipy (con dendrogramma) e sklearn (più diretto).

In [ ]:
# ============================================
# DEMO 5: sklearn vs scipy
# ============================================

print("="*70)
print("DEMO 5: Due Modi per Fare Clustering Gerarchico")
print("="*70)

# Usiamo i dati multi-cluster
print("\n📊 Dataset: 100 punti, 4 cluster veri")

# ============================================
# APPROCCIO 1: scipy (con dendrogramma)
# ============================================
print("\n" + "="*70)
print("APPROCCIO 1: scipy.cluster.hierarchy")
print("="*70)

from scipy.cluster.hierarchy import linkage, fcluster

# Passo 1: Calcola linkage
Z = linkage(X_multi_scaled, method='ward')

# Passo 2: Taglia a K cluster
labels_scipy = fcluster(Z, t=4, criterion='maxclust')

# Nota: fcluster numera da 1, non da 0
labels_scipy = labels_scipy - 1  # Convertiamo a 0-indexed

sil_scipy = silhouette_score(X_multi_scaled, labels_scipy)
print(f"""
Codice:
  Z = linkage(X_scaled, method='ward')
  labels = fcluster(Z, t=4, criterion='maxclust')

Pro:
  ✓ Accesso al dendrogramma
  ✓ Puoi esplorare diversi K senza ricalcolare
  ✓ Più flessibile

Contro:
  ✗ Due passi separati
  ✗ API meno intuitiva

Silhouette: {sil_scipy:.3f}
""")

# ============================================
# APPROCCIO 2: sklearn AgglomerativeClustering
# ============================================
print("="*70)
print("APPROCCIO 2: sklearn.cluster.AgglomerativeClustering")
print("="*70)

model = AgglomerativeClustering(n_clusters=4, linkage='ward')
labels_sklearn = model.fit_predict(X_multi_scaled)

sil_sklearn = silhouette_score(X_multi_scaled, labels_sklearn)
print(f"""
Codice:
  model = AgglomerativeClustering(n_clusters=4, linkage='ward')
  labels = model.fit_predict(X_scaled)

Pro:
  ✓ API coerente con altri modelli sklearn
  ✓ Singola chiamata
  ✓ Integrazione pipeline

Contro:
  ✗ Nessun dendrogramma diretto
  ✗ Devi specificare n_clusters a priori

Silhouette: {sil_sklearn:.3f}
""")

# ============================================
# Verifica che diano gli stessi risultati
# ============================================
print("="*70)
print("VERIFICA EQUIVALENZA")
print("="*70)

# Nota: le label potrebbero essere permutate diversamente
from sklearn.metrics import adjusted_rand_score
ari = adjusted_rand_score(labels_scipy, labels_sklearn)
print(f"\nAdjusted Rand Index tra i due metodi: {ari:.3f}")
print("(1.0 = clustering identici, anche se le etichette sono diverse)")

# ============================================
# Quando usare quale?
# ============================================
print("\n" + "="*70)
print("QUANDO USARE QUALE?")
print("="*70)
print("""
📌 USA scipy QUANDO:
   • Vuoi esplorare il dendrogramma
   • Non sai quanti cluster vuoi
   • Vuoi provare diversi K senza ricalcolare

📌 USA sklearn QUANDO:
   • Sai già quanti cluster vuoi
   • Hai bisogno di integrazione con Pipeline
   • Vuoi API coerente con altri modelli
""")

---

## 4. Esercizi

### 📝 Esercizio 22.1 — Analisi Completa con Dendrogramma

**Consegna:**
Hai dati di pazienti con 3 misurazioni mediche. Esegui un'analisi gerarchica completa:

1. Crea il dendrogramma con Ward linkage
2. Identifica il numero ottimale di cluster dalle "gambe lunghe"
3. Estrai i cluster e calcola la silhouette
4. Interpreta i profili dei cluster

**Dataset:**
```python
eta = [25, 30, 28, 65, 70, 68, 45, 48, 50, 22, 75, 42]
pressione = [120, 125, 118, 145, 155, 150, 130, 135, 132, 115, 160, 128]
colesterolo = [180, 190, 175, 240, 260, 250, 210, 215, 205, 170, 270, 200]
```

In [ ]:
# ============================================
# ESERCIZIO 22.1 — SOLUZIONE
# ============================================

import pandas as pd

print("="*70)
print("ESERCIZIO 22.1 — Clustering Gerarchico di Dati Medici")
print("="*70)

# ============================================
# PASSO 1: Preparazione dati
# ============================================
eta = [25, 30, 28, 65, 70, 68, 45, 48, 50, 22, 75, 42]
pressione = [120, 125, 118, 145, 155, 150, 130, 135, 132, 115, 160, 128]
colesterolo = [180, 190, 175, 240, 260, 250, 210, 215, 205, 170, 270, 200]

df_pazienti = pd.DataFrame({
    'paziente': [f'P{i}' for i in range(1, 13)],
    'eta': eta,
    'pressione': pressione,
    'colesterolo': colesterolo
})

print("\n📊 Dataset:")
print(df_pazienti)

# Scaling
X_pazienti = df_pazienti[['eta', 'pressione', 'colesterolo']].values
scaler = StandardScaler()
X_pazienti_scaled = scaler.fit_transform(X_pazienti)

# ============================================
# PASSO 2: Dendrogramma
# ============================================
print("\n" + "="*70)
print("PASSO 2: Dendrogramma")
print("="*70)

Z = linkage(X_pazienti_scaled, method='ward')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Dendrogramma con etichette pazienti
dendrogram(Z, ax=axes[0], labels=df_pazienti['paziente'].values,
           leaf_rotation=45, leaf_font_size=10)
axes[0].set_title('Dendrogramma - Pazienti', fontsize=12)
axes[0].set_xlabel('Paziente')
axes[0].set_ylabel('Distanza (Ward)')

# Aggiungiamo linee di taglio
axes[0].axhline(y=4, color='red', linestyle='--', linewidth=2, label='K=3')
axes[0].axhline(y=2.5, color='orange', linestyle='--', linewidth=2, label='K=4')
axes[0].legend()

# Distanze di fusione
distances = Z[:, 2]
diff_distances = np.diff(distances)

axes[1].bar(range(len(diff_distances)), diff_distances, alpha=0.7, color='steelblue')
axes[1].set_xlabel('Indice fusione')
axes[1].set_ylabel('Salto nelle distanze')
axes[1].set_title('Salti nelle distanze\n(il più alto suggerisce K)')
axes[1].grid(True, alpha=0.3)

# Evidenzia il salto maggiore
max_jump_idx = np.argmax(diff_distances)
axes[1].bar(max_jump_idx, diff_distances[max_jump_idx], color='red', alpha=0.8)
axes[1].annotate(f'Salto max\nK={len(X_pazienti)-max_jump_idx-1}', 
                  xy=(max_jump_idx, diff_distances[max_jump_idx]),
                  xytext=(max_jump_idx+1, diff_distances[max_jump_idx]*1.1),
                  fontsize=10)

plt.tight_layout()
plt.show()

# ============================================
# PASSO 3: Estrazione cluster
# ============================================
print("\n" + "="*70)
print("PASSO 3: Estrazione Cluster")
print("="*70)

# Dal dendrogramma, K=3 sembra ottimale
K_ottimale = 3
labels = fcluster(Z, t=K_ottimale, criterion='maxclust') - 1  # 0-indexed

df_pazienti['cluster'] = labels

sil = silhouette_score(X_pazienti_scaled, labels)
print(f"\n🎯 Numero cluster scelto: K={K_ottimale}")
print(f"📈 Silhouette Score: {sil:.3f}")

# ============================================
# PASSO 4: Interpretazione
# ============================================
print("\n" + "="*70)
print("PASSO 4: Interpretazione dei Cluster")
print("="*70)

cluster_profiles = df_pazienti.groupby('cluster')[['eta', 'pressione', 'colesterolo']].mean()
print("\n📊 Profili medi per cluster:")
print(cluster_profiles.round(1))

print("\n📌 INTERPRETAZIONE:")
for cluster_id in sorted(df_pazienti['cluster'].unique()):
    cluster_data = df_pazienti[df_pazienti['cluster'] == cluster_id]
    eta_media = cluster_data['eta'].mean()
    pressione_media = cluster_data['pressione'].mean()
    colesterolo_medio = cluster_data['colesterolo'].mean()
    
    print(f"\n   CLUSTER {cluster_id} ({len(cluster_data)} pazienti):")
    print(f"   Età media: {eta_media:.0f}, Pressione: {pressione_media:.0f}, Colesterolo: {colesterolo_medio:.0f}")
    
    if eta_media < 35:
        print("   → PROFILO: 🟢 Giovani a basso rischio")
    elif eta_media > 60:
        print("   → PROFILO: 🔴 Anziani ad alto rischio cardiovascolare")
    else:
        print("   → PROFILO: 🟡 Mezza età, rischio moderato")

# Visualizzazione finale
fig, ax = plt.subplots(figsize=(8, 6))
scatter = ax.scatter(df_pazienti['eta'], df_pazienti['colesterolo'], 
                      c=df_pazienti['cluster'], cmap='viridis', s=100, 
                      edgecolors='black', alpha=0.8)
for i, row in df_pazienti.iterrows():
    ax.annotate(row['paziente'], (row['eta']+1, row['colesterolo']+3), fontsize=8)
ax.set_xlabel('Età')
ax.set_ylabel('Colesterolo')
ax.set_title(f'Clustering Gerarchico (K={K_ottimale}, Silhouette={sil:.3f})')
plt.colorbar(scatter, label='Cluster')
ax.grid(True, alpha=0.3)
plt.show()

---

### 📝 Esercizio 22.2 — Confronto Metodi di Linkage

**Consegna:** Dato un dataset di 60 punti con cluster di forme diverse, confronta i 4 metodi di linkage.

**Richieste:**
1. Genera un dataset con 3 cluster di forme diverse (uno allungato, uno compatto, uno sparso)
2. Applica clustering gerarchico con tutti e 4 i metodi di linkage
3. Calcola e confronta i Silhouette Score
4. Identifica quale metodo funziona meglio e perché

In [ ]:
# ============================================
# ESERCIZIO 22.2 — SOLUZIONE
# ============================================

print("="*70)
print("ESERCIZIO 22.2 — Confronto Metodi di Linkage")
print("="*70)

# ============================================
# PASSO 1: Dataset con cluster di forme diverse
# ============================================
np.random.seed(42)

# Cluster 1: Allungato (ellittico)
n1 = 20
cluster1 = np.column_stack([
    np.random.normal(0, 0.5, n1),
    np.random.normal(0, 2.5, n1)  # Più allungato sull'asse y
])

# Cluster 2: Compatto (sferico)
n2 = 20
cluster2 = np.random.normal(loc=[5, 0], scale=0.5, size=(n2, 2))

# Cluster 3: Sparso
n3 = 20
cluster3 = np.random.normal(loc=[2.5, 5], scale=1.5, size=(n3, 2))

X_forme = np.vstack([cluster1, cluster2, cluster3])
y_true = np.array([0]*n1 + [1]*n2 + [2]*n3)

# Scaling
scaler = StandardScaler()
X_forme_scaled = scaler.fit_transform(X_forme)

print(f"📊 Dataset: {len(X_forme)} punti, 3 cluster di forme diverse")
print("   - Cluster 0: Allungato (ellittico)")
print("   - Cluster 1: Compatto (sferico)")
print("   - Cluster 2: Sparso")

# ============================================
# PASSO 2: Confronto dei 4 metodi di linkage
# ============================================
print("\n" + "="*70)
print("PASSO 2: Confronto Metodi di Linkage")
print("="*70)

metodi = ['single', 'complete', 'average', 'ward']
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

risultati = {}

for idx, metodo in enumerate(metodi):
    # Dendrogramma
    Z = linkage(X_forme_scaled, method=metodo)
    dendrogram(Z, ax=axes[0, idx], truncate_mode='lastp', p=10, 
               leaf_rotation=45, no_labels=True)
    axes[0, idx].set_title(f'{metodo.upper()}', fontsize=12, fontweight='bold')
    if idx == 0:
        axes[0, idx].set_ylabel('Distanza')
    
    # Clustering con K=3
    labels = fcluster(Z, t=3, criterion='maxclust') - 1
    
    # Calcola silhouette
    sil = silhouette_score(X_forme_scaled, labels)
    ari = adjusted_rand_score(y_true, labels)
    risultati[metodo] = {'silhouette': sil, 'ari': ari, 'labels': labels}
    
    # Scatter plot
    scatter = axes[1, idx].scatter(X_forme[:, 0], X_forme[:, 1], 
                                    c=labels, cmap='viridis', s=50, alpha=0.7)
    axes[1, idx].set_title(f'Sil={sil:.3f}, ARI={ari:.3f}', fontsize=10)
    if idx == 0:
        axes[1, idx].set_ylabel('y')
    axes[1, idx].set_xlabel('x')
    axes[1, idx].grid(True, alpha=0.3)

plt.suptitle('Confronto Metodi di Linkage su Cluster di Forme Diverse', fontsize=14)
plt.tight_layout()
plt.show()

# ============================================
# PASSO 3: Tabella comparativa
# ============================================
print("\n" + "="*70)
print("PASSO 3: Tabella Comparativa")
print("="*70)

print("\n" + "-"*50)
print(f"{'Metodo':<12} {'Silhouette':>12} {'ARI':>10}")
print("-"*50)
for metodo in metodi:
    sil = risultati[metodo]['silhouette']
    ari = risultati[metodo]['ari']
    marker = "⭐" if metodo == max(risultati, key=lambda x: risultati[x]['silhouette']) else "  "
    print(f"{marker}{metodo:<10} {sil:>12.3f} {ari:>10.3f}")
print("-"*50)

# ============================================
# PASSO 4: Analisi e Conclusione
# ============================================
best_method = max(risultati, key=lambda x: risultati[x]['silhouette'])
worst_method = min(risultati, key=lambda x: risultati[x]['silhouette'])

print("\n" + "="*70)
print("PASSO 4: Analisi e Conclusione")
print("="*70)

print(f"""
📊 ANALISI DEI RISULTATI:

🏆 MIGLIOR METODO: {best_method.upper()}
   - Silhouette: {risultati[best_method]['silhouette']:.3f}
   - ARI: {risultati[best_method]['ari']:.3f}

❌ PEGGIOR METODO: {worst_method.upper()}
   - Silhouette: {risultati[worst_method]['silhouette']:.3f}
   - ARI: {risultati[worst_method]['ari']:.3f}

📌 PERCHÉ?
   - WARD funziona bene perché minimizza la varianza intra-cluster
   - SINGLE soffre dell'effetto catena con cluster allungati
   - COMPLETE può separare artificialmente cluster elongati
   - AVERAGE è un buon compromesso ma meno robusto di Ward
""")

---

### 📝 Esercizio 22.3 — sklearn AgglomerativeClustering

**Consegna:** Usa `sklearn.cluster.AgglomerativeClustering` per segmentare clienti di un e-commerce.

**Dataset:**
```python
spesa_media = [50, 55, 48, 200, 250, 230, 120, 130, 125, 45, 280, 115]
frequenza_acquisti = [2, 3, 2, 8, 10, 9, 5, 6, 5, 2, 12, 4]
```

**Richieste:**
1. Usa AgglomerativeClustering di sklearn con linkage Ward
2. Prova K=2, 3, 4 e calcola Silhouette per ognuno
3. Scegli il K ottimale e interpreta i cluster
4. Assegna nomi descrittivi ai cluster (es: "Clienti Premium", "Occasionali", ecc.)

In [ ]:
# ============================================
# ESERCIZIO 22.3 — SOLUZIONE
# ============================================
from sklearn.cluster import AgglomerativeClustering

print("="*70)
print("ESERCIZIO 22.3 — sklearn AgglomerativeClustering")
print("="*70)

# ============================================
# PASSO 1: Preparazione dati
# ============================================
spesa_media = [50, 55, 48, 200, 250, 230, 120, 130, 125, 45, 280, 115]
frequenza_acquisti = [2, 3, 2, 8, 10, 9, 5, 6, 5, 2, 12, 4]

df_clienti = pd.DataFrame({
    'cliente': [f'C{i}' for i in range(1, 13)],
    'spesa_media': spesa_media,
    'frequenza_acquisti': frequenza_acquisti
})

print("\n📊 Dataset Clienti E-commerce:")
print(df_clienti)

X_clienti = df_clienti[['spesa_media', 'frequenza_acquisti']].values
scaler = StandardScaler()
X_clienti_scaled = scaler.fit_transform(X_clienti)

# ============================================
# PASSO 2: Test K=2, 3, 4 con AgglomerativeClustering
# ============================================
print("\n" + "="*70)
print("PASSO 2: Test K=2, 3, 4")
print("="*70)

K_range = [2, 3, 4]
risultati_k = {}

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, k in enumerate(K_range):
    # Usa AgglomerativeClustering di sklearn
    agg = AgglomerativeClustering(n_clusters=k, linkage='ward')
    labels = agg.fit_predict(X_clienti_scaled)
    
    sil = silhouette_score(X_clienti_scaled, labels)
    risultati_k[k] = {'silhouette': sil, 'labels': labels}
    
    # Plot
    scatter = axes[idx].scatter(df_clienti['spesa_media'], 
                                 df_clienti['frequenza_acquisti'],
                                 c=labels, cmap='viridis', s=100, 
                                 edgecolors='black', alpha=0.8)
    for i, row in df_clienti.iterrows():
        axes[idx].annotate(row['cliente'], 
                           (row['spesa_media']+5, row['frequenza_acquisti']+0.2),
                           fontsize=8)
    axes[idx].set_xlabel('Spesa Media (€)')
    axes[idx].set_ylabel('Frequenza Acquisti')
    axes[idx].set_title(f'K={k}, Silhouette={sil:.3f}', fontsize=12)
    axes[idx].grid(True, alpha=0.3)

plt.suptitle('AgglomerativeClustering (Ward) - Confronto K', fontsize=14)
plt.tight_layout()
plt.show()

# Tabella risultati
print("\n📊 Risultati per ogni K:")
print("-"*40)
print(f"{'K':>5} {'Silhouette':>15}")
print("-"*40)
for k in K_range:
    sil = risultati_k[k]['silhouette']
    marker = "⭐" if k == max(risultati_k, key=lambda x: risultati_k[x]['silhouette']) else "  "
    print(f"{marker}{k:>3} {sil:>15.3f}")
print("-"*40)

# ============================================
# PASSO 3: K ottimale e interpretazione
# ============================================
K_ottimale = max(risultati_k, key=lambda x: risultati_k[x]['silhouette'])
print(f"\n🎯 K OTTIMALE: {K_ottimale}")

# Applica clustering finale
df_clienti['cluster'] = risultati_k[K_ottimale]['labels']

print("\n" + "="*70)
print("PASSO 3: Interpretazione Cluster")
print("="*70)

# Profili
cluster_profiles = df_clienti.groupby('cluster')[['spesa_media', 'frequenza_acquisti']].mean()
print("\n📊 Profili medi per cluster:")
print(cluster_profiles.round(1))

# ============================================
# PASSO 4: Nomi descrittivi
# ============================================
print("\n" + "="*70)
print("PASSO 4: Assegnazione Nomi Cluster")
print("="*70)

nomi_cluster = {}
for cluster_id in sorted(df_clienti['cluster'].unique()):
    cluster_data = df_clienti[df_clienti['cluster'] == cluster_id]
    spesa_media_cluster = cluster_data['spesa_media'].mean()
    freq_media = cluster_data['frequenza_acquisti'].mean()
    
    # Logica di naming basata sui valori
    if spesa_media_cluster > 180 and freq_media > 7:
        nome = "🏆 Premium VIP"
    elif spesa_media_cluster > 100 and freq_media > 4:
        nome = "🌟 Fedeli Standard"
    else:
        nome = "🛒 Occasionali"
    
    nomi_cluster[cluster_id] = nome
    
    print(f"\n   CLUSTER {cluster_id} → {nome}")
    print(f"   Spesa media: €{spesa_media_cluster:.0f}, Frequenza: {freq_media:.1f}")
    print(f"   Clienti: {', '.join(cluster_data['cliente'].values)}")

# Visualizzazione finale con nomi
fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.viridis(np.linspace(0, 1, K_ottimale))

for cluster_id in sorted(df_clienti['cluster'].unique()):
    mask = df_clienti['cluster'] == cluster_id
    ax.scatter(df_clienti.loc[mask, 'spesa_media'], 
               df_clienti.loc[mask, 'frequenza_acquisti'],
               c=[colors[cluster_id]], s=150, edgecolors='black',
               label=nomi_cluster[cluster_id], alpha=0.8)

for i, row in df_clienti.iterrows():
    ax.annotate(row['cliente'], (row['spesa_media']+5, row['frequenza_acquisti']+0.2),
                fontsize=9)

ax.set_xlabel('Spesa Media (€)', fontsize=11)
ax.set_ylabel('Frequenza Acquisti', fontsize=11)
ax.set_title('Segmentazione Clienti E-commerce\nClustering Gerarchico (Ward)', fontsize=13)
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ Segmentazione completata!")

---

## 🎯 5. Conclusione

### ✅ Cosa Portarsi a Casa

| Concetto | Cosa Ricordare |
|----------|----------------|
| **Dendrogramma** | Visualizza la gerarchia di fusioni; il taglio orizzontale determina K |
| **Linkage Ward** | Default sicuro, minimizza varianza intra-cluster |
| **Single Linkage** | Soggetto all'effetto catena - evitare con cluster allungati |
| **fcluster** | Per tagliare il dendrogramma e ottenere le etichette |
| **scipy vs sklearn** | scipy dà dendrogramma, sklearn è più semplice ma senza gerarchia |

### ⚠️ Errori Comuni

| Errore | Perché è Sbagliato | Correzione |
|--------|-------------------|------------|
| Usare single linkage sempre | Effetto catena crea cluster innaturali | Preferire Ward o average |
| Non scalare i dati | Distanze influenzate da scale diverse | Sempre StandardScaler |
| Ignorare il dendrogramma | Perdi informazione sulla struttura | Analizza sempre prima di tagliare |
| Scegliere K guardando solo Silhouette | Può ignorare struttura gerarchica | Combina con analisi dendrogramma |

### 🔗 Ponte verso la Lezione 23

Nella prossima lezione esploreremo **DBSCAN** (Density-Based Spatial Clustering):
- **Non richiede K** a priori
- Trova cluster di **forma arbitraria**
- Identifica automaticamente **outliers/anomalie**
- Basato su densità invece che distanza

---

## 📚 6. Bignami — Clustering Gerarchico

### 📖 Definizioni Chiave

| Termine | Definizione |
|---------|-------------|
| **Agglomerativo** | Bottom-up: ogni punto inizia come cluster, poi fusioni successive |
| **Divisivo** | Top-down: tutti i punti in un cluster, poi divisioni successive |
| **Dendrogramma** | Albero che mostra la gerarchia delle fusioni/divisioni |
| **Linkage** | Criterio per misurare la distanza tra cluster |
| **Single Linkage** | Distanza minima tra punti dei due cluster |
| **Complete Linkage** | Distanza massima tra punti dei due cluster |
| **Average Linkage** | Media delle distanze tra tutti i punti |
| **Ward Linkage** | Minimizza l'aumento di varianza intra-cluster |
| **Effetto Catena** | Problema di single linkage: crea cluster allungati innaturali |

### 📐 Formule

| Formula | Significato |
|---------|-------------|
| $d_{single}(A, B) = \min_{a \in A, b \in B} d(a, b)$ | Linkage Single |
| $d_{complete}(A, B) = \max_{a \in A, b \in B} d(a, b)$ | Linkage Complete |
| $d_{average}(A, B) = \frac{1}{\|A\| \cdot \|B\|} \sum_{a \in A} \sum_{b \in B} d(a, b)$ | Linkage Average |
| $d_{ward}(A, B) = \sqrt{\frac{2\|A\|\|B\|}{\|A\|+\|B\|}} \|\bar{a} - \bar{b}\|$ | Linkage Ward |

### ✅ Checklist Pre-Clustering Gerarchico

```
□ Dati scalati con StandardScaler?
□ Scelto il metodo di linkage appropriato? (Ward = default sicuro)
□ Generato il dendrogramma per visualizzare la struttura?
□ Analizzati i salti nelle distanze per scegliere K?
□ Tagliato con fcluster usando il K scelto?
□ Calcolato Silhouette Score?
□ Interpretati i cluster risultanti?
```

### 💻 Template di Codice

```python
# === CLUSTERING GERARCHICO CON SCIPY ===
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# 1. Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. Linkage matrix
Z = linkage(X_scaled, method='ward')  # o 'single', 'complete', 'average'

# 3. Dendrogramma
plt.figure(figsize=(10, 5))
dendrogram(Z, truncate_mode='lastp', p=20)
plt.title('Dendrogramma')
plt.xlabel('Campioni')
plt.ylabel('Distanza')
plt.show()

# 4. Taglio e cluster
K = 3
labels = fcluster(Z, t=K, criterion='maxclust') - 1

# 5. Valutazione
sil = silhouette_score(X_scaled, labels)
print(f"Silhouette Score: {sil:.3f}")

# === CON SKLEARN ===
from sklearn.cluster import AgglomerativeClustering

agg = AgglomerativeClustering(n_clusters=K, linkage='ward')
labels = agg.fit_predict(X_scaled)
```

### 🎯 Quando Usare

| Usa Gerarchico quando... | Evita Gerarchico quando... |
|--------------------------|---------------------------|
| Vuoi esplorare la struttura dei dati | Hai >10,000 campioni (lento) |
| Non sai K a priori | Hai già K definito |
| Vuoi un dendrogramma interpretabile | Vuoi solo le etichette rapidamente |
| I cluster hanno struttura gerarchica naturale | I cluster sono equi-distribuiti |

---

✅ **Lezione 22 Completata!**